# Cap.17 - **Trabalhando com APIs**

Neste capítulo aprenderemos a **escrever um programa autocontido para gerar uma visualização baseada em dados recuperados pelo programa. Seu programa usará uma <code>API (Application Programming Interface, ou Interface de Programação de Aplicativos)</code> web para solicitar informações específicas de um site automaticamente, em vez de pedir páginas inteiras. Então essas informações serão usadas para gerar uma visualização.** Como os programas escritos dessa forma sempre usarão dados atuais para gerar uma visualização, mesmo que esses dados mudem rapidamente, eles estarão sempre atualizados.

## Usando uma **<code>API web</code>**

Uma **<code>API web</code> é uma parte de um site projetada para interagir com programas que usam <code>URLs</code> bem específicos a fim de requisitar determinadas informações.** Esse tipo de requisição é conhecido como chamada de <code>**API**</code>. Os dados solicitados serão devolvidos em um formato facilmente processável, por exemplo, <code>**JSON ou CSV**</code>. **A maioria das aplicações que depende de fontes de dados externas, como aquelas que se integram a sites de mídias sociais**, contam com chamadas de <code>**API**</code>.

### Git e GitHub

Nossa visualização será baseada em informações do <code>**GitHub**</code> : **um site que permite aos programadores colaborar em projetos. Usaremos a <code>API do GitHub</code>  para solicitar informações do site sobre projetos Python e então vamos gerar uma visualização interativa da popularidade relativa desses projetos no <code>Pygal</code>.**     

O <code>**GitHub**</code>  (https://github.com/) tem esse nome por causa do <code>**Git**</code>  – **um sistema distribuído de controle de versões que permite às equipes de programadores colaborar em projetos. O <code>Git</code>  ajuda as pessoas a administrar seus trabalhos individuais em um projeto, de modo que as alterações feitas por uma pessoa não interfiram nas mudanças que outras pessoas estão fazendo.** Ao implementar um novo recurso em um projeto, o <code>**Git**</code>  controlará as alterações que você fizer em cada arquivo.

**Quando seu novo código estiver funcionando, você fará <code>commit</code> das alterações feitas e o <code>Git</code> registrará o novo estado de seu projeto.** Se você cometer um erro e quiser reverter suas alterações, poderá facilmente voltar para qualquer estado anterior funcional. (Para saber mais sobre controle de versões com o <code>**Git**</code> , consulte o Apêndice D.) **Os projetos no <code>GitHub</code>  são armazenados em repositórios, que contêm tudo que está associado ao projeto: seu código, informações sobre os colaboradores, qualquer relatório de problema ou bug, e assim por diante.**

Quando os usuários do <code>**GitHub**</code>  gostam de um projeto, eles podem lhe conceder uma “estrela” para mostrar seu apoio e monitorar os projetos que queiram usar. Neste capítulo **escreveremos um programa para fazer download automático de informações sobre os projetos Python com mais estrelas no <code>GitHub</code>  e, em seguida, criaremos uma visualização informativa desses projetos.**

### Requisitando dados usando uma chamada de <code>**API**</code>

**A <code>API do GitHub</code> permite requisitar várias informações por meio de chamadas de <code>API</code>**. Para ver como é a aparência de uma chamada de <code>**API**</code>, digite o seguinte na barra de endereço de seu navegador e tecle **ENTER**:
https://api.github.com/search/repositories? q=language:python&sort=stars

**Essa chamada devolve o número de projetos Python hospedados no <code>**GitHub**</code> no momento, bem como informações sobre os repositórios Python mais populares.** Vamos analisar a chamada. **A primeira parte, https://api.github.com/, direciona a requisição para a parte do site do <code>**GitHub**</code> que responde a chamadas de <code>API</code>**. A próxima parte, search/repositories, diz à <code>**API**</code> para conduzir uma pesquisa em todos os repositórios do <code>**GitHub**</code>.

**O ponto de interrogação depois de <code>repositories</code> indica que estamos prestes a passar um argumento. A letra <code>q</code> quer dizer <code>query</code> e o sinal de igualdade nos permite começar a especificá-la <code>(q=)</code>. Ao usar <code>language:python</code>, sinalizamos que queremos informações somente sobre os <code>repositórios</code> que tenham Python como a linguagem principal.** A última parte, <code>**&sort=stars**</code>, ordena os projetos de acordo com o número de estrelas que receberam.

**O trecho a seguir mostra as primeiras linhas da resposta**. Ao observá- la, podemos notar que esse <code>**URL**</code> não tem como propósito ser usado por seres humanos.

In [ ]:
{"total_count": 713062, "incomplete_results": false, "items": [
{"id": 3544424, "name": "httpie", "full_name": "jkbrzt/httpie", --trecho omitido--

Como podemos ver na segunda linha da saída, o **<code>GitHub</code> encontrou um total de 713.062 projetos Python na ocasião em que testamos isso. Como o valor de <code>"incomplete_results" é false</code>, sabemos que a requisição foi bem-sucedida (não está incompleta).** Se o <code>**GitHub**</code> não tivesse sido capaz de processar totalmente a requisição da <code>**API**</code>, ele teria devolvido <code>**true**</code> aqui. Os **<code>"items"</code>** devolvidos são exibidos na lista que está na sequência, a qual contém detalhes sobre os projetos Python mais populares no <code>**GitHub**</code>.

### Instalando o pacote requests

**O pacote <code>requests</code> permite que um programa Python solicite facilmente informações a um site e analise a resposta devolvida.** Para instalar esse pacote, execute um comando como este: <code>**$ pip install --user requests**</code> **Se você ainda não usou o <code>pip</code>, consulte a seção “Instalando pacotes Python com o <code>pip</code>”**. (Talvez você precise usar uma versão um pouco diferente desse comando, conforme a configuração de seu sistema.)

### Processando uma resposta de <code>**API**</code>

Agora começaremos a escrever um programa para fazer uma chamada de <code>**API**</code> e processar o resultado identificando os projetos Python com mais estrelas no <code>**GitHub**</code>:

In [26]:
#python_repos.py 
import requests

# Faz uma chamada de API e armazena a resposta 
url = 'https://api.github.com/search/repositories?q=language:python&sort=stars'
r = requests.get(url) 
print("Status code:", r.status_code)

# Armazena a resposta da API em uma variável 
response_dict = r.json() 

# Processa o resultado 
print(response_dict.keys())

Status code: 200
dict_keys(['total_count', 'incomplete_results', 'items'])


E **importamos o módulo <code>requests</code>. E armazenamos o <code>URL</code> da chamada da <code>API</code> e então usamos <code>requests</code> para fazer a chamada. Chamamos <code>get()</code>, passamos o <code>URL</code> e armazenamos o objeto com a resposta na variável <code>r</code>. O objeto com a resposta tem um atributo chamado <code>status_code</code>, que nos informa se a requisição foi bem-sucedida.** (Um código de status igual a 200 indica sucesso na resposta.) E exibimos o valor de <code>**status_code**</code> para garantir que a chamada foi realizada com sucesso.

**A <code>API</code> devolve as informações em formato <code>JSON</code>, portanto usamos o método <code>json()</code> para convertê-las em um <code>dicionário</code> Python**. Armazenamos o <code>**dicionário**</code> resultante em <code>**response_dict**</code>.

**NOTA**    
**Chamadas simples como essa devem devolver um conjunto completo de resultados, portanto é seguro ignorar o valor associado a <code>'incomplete_results'</code>. Porém, quando fizer chamadas de <code>API</code> mais complexas, seu programa deverá conferir esse valor.**


### Trabalhando com o dicionário de resposta

**Agora que temos a informação da chamada de <code>API</code> na forma de um dicionário, podemos trabalhar com os dados armazenados nele. Vamos gerar uma saída que sintetize as informações.** Essa é uma boa maneira de garantir que recebemos as informações esperadas e começar a analisar os dados em que estamos interessados: 

In [37]:
#python_repos.py
import requests

# Faz uma chamada de API e armazena a resposta 
url = 'https://api.github.com/search/repositories?q=language:python&sort=stars' 
r = requests.get(url) 
print("Status code:", r.status_code)

# Armazena a resposta da API em uma variável 
response_dict = r.json() 
print("Total repositories:", response_dict['total_count'])

# Explora informações sobre os repositórios  
repo_dicts = response_dict['items']
print("Repositories returned:", len(repo_dicts))

# Analisa o primeiro repositório 
repo_dict = repo_dicts[0]
print("\nKeys:", len(repo_dict)) 
for key in sorted(repo_dict.keys()): 
    print(key)

Status code: 200
Total repositories: 15361239
Repositories returned: 30

Keys: 80
allow_forking
archive_url
archived
assignees_url
blobs_url
branches_url
clone_url
collaborators_url
comments_url
commits_url
compare_url
contents_url
contributors_url
created_at
default_branch
deployments_url
description
disabled
downloads_url
events_url
fork
forks
forks_count
forks_url
full_name
git_commits_url
git_refs_url
git_tags_url
git_url
has_discussions
has_downloads
has_issues
has_pages
has_projects
has_wiki
homepage
hooks_url
html_url
id
is_template
issue_comment_url
issue_events_url
issues_url
keys_url
labels_url
language
languages_url
license
merges_url
milestones_url
mirror_url
name
node_id
notifications_url
open_issues
open_issues_count
owner
private
pulls_url
pushed_at
releases_url
score
size
ssh_url
stargazers_count
stargazers_url
statuses_url
subscribers_url
subscription_url
svn_url
tags_url
teams_url
topics
trees_url
updated_at
url
visibility
watchers
watchers_count
web_commit_signoff_re

**E exibimos o valor associado a <code>'total_count'</code>, que representa o número total de <code>repositórios</code> Python no <code>GitHub</code>.
O valor associado a <code>'items'</code> é uma lista que contém vários dicionários, cada um contendo dados sobre um repositório Python individual.** E armazenamos essa lista de dicionários em <code>**repo_dicts**</code>. Então exibimos o tamanho de <code>**repo_dicts**</code> para ver o número de <code>**repositórios**</code> para os quais temos informações.

**Para observar melhor as informações devolvidas sobre cada <code>repositório</code>, extraímos o primeiro <code>'items</code> de <code>repositório</code> e o armazenamos em <code>repo_dict</code>. Então exibimos a quantidade de chaves do dicionário para ver quantas informações temos.** E exibimos todas as <code>**chaves (keys)**</code> do dicionário para ver quais tipos de informação estão incluídos.
O resultado começa a nos dar uma imagem mais clara dos dados propriamente ditos:    
**Status code: 200   
Total repositories: 713062     
Repositories returned: 30**   


**A <code>API do GitHub</code> devolve muitas informações sobre cada repositório: há 68 chaves em <code>repo_dict</code>. Ao observar essas chaves, você terá uma noção do tipo de informação que pode ser extraído a respeito de um projeto.** (A única maneira de saber quais informações estão disponíveis por meio de uma <code>**API**</code> é ler a documentação ou analisar as informações por meio de código, como estamos fazendo nesse caso.) Vamos extrair os valores de algumas das chaves em <code>**repo_dict**</code>:

In [46]:
#python_repos.py
import requests

# Faz uma chamada de API e armazena a resposta 
url = 'https://api.github.com/search/repositories?q=language:python&sort=stars' 
r = requests.get(url) 
print("Status code:", r.status_code)

# Armazena a resposta da API em uma variável 
response_dict = r.json() 
print("Total repositories:", response_dict['total_count'])

# Explora informações sobre os repositórios  
repo_dicts = response_dict['items']
print("Repositories returned:", len(repo_dicts))

# Analisa o primeiro repositório 
repo_dict = repo_dicts[0]
print("\nSelected information about first repository:") 
print('Name:', repo_dict['name']) 
print('Owner:', repo_dict['owner']['login']) 
print('Stars:', repo_dict['stargazers_count']) 
print('Repository:', repo_dict['html_url']) 
print('Created:', repo_dict['created_at']) 
print('Updated:', repo_dict['updated_at']) 
print('Description:', repo_dict['description'])

for key in sorted(repo_dict.keys()): 
    print(key)

Status code: 200
Total repositories: 16125622
Repositories returned: 30

Selected information about first repository:
Name: system-design-primer
Owner: donnemartin
Stars: 266405
Repository: https://github.com/donnemartin/system-design-primer
Created: 2017-02-26T16:15:28Z
Updated: 2024-08-08T05:31:05Z
Description: Learn how to design large-scale systems. Prep for the system design interview.  Includes Anki flashcards.
allow_forking
archive_url
archived
assignees_url
blobs_url
branches_url
clone_url
collaborators_url
comments_url
commits_url
compare_url
contents_url
contributors_url
created_at
default_branch
deployments_url
description
disabled
downloads_url
events_url
fork
forks
forks_count
forks_url
full_name
git_commits_url
git_refs_url
git_tags_url
git_url
has_discussions
has_downloads
has_issues
has_pages
has_projects
has_wiki
homepage
hooks_url
html_url
id
is_template
issue_comment_url
issue_events_url
issues_url
keys_url
labels_url
language
languages_url
license
merges_url
milesto

Nesse exemplo exibimos os valores de diversas <code>chaves do dicionário</code> do primeiro repositório. E exibimos o nome do projeto. **Um <code>dicionário completo</code> representa o dono do projeto; assim, usamos a chave <code>owner</code> para acessar o dicionário que o representa e então usamos a chave <code>login</code> para obter o seu nome de <code>login</code>. E exibimos a quantidade de estrelas que o projeto recebeu e o <code>URL do repositório do projeto no GitHub</code>.** Em seguida, mostramos a data em que o projeto foi criado e quando foi atualizado pela última vez. Por fim, exibimos a descrição do repositório;

**Podemos ver que o projeto Python com mais estrelas no <code>GitHub</code> (na época em que esta obra foi escrita) é o <code>HTTPie</code>, cujo proprietário é o usuário <code>jkbrzt</code>, e recebeu estrelas de mais de 16 mil usuários do <code>GitHub</code>.** Vemos também o <code>**URL do repositório do projeto**</code>, a data de sua criação – fevereiro de 2012 – e que o projeto foi atualizado recentemente. Por fim, a descrição nos informa que o <code>HTTPie</code> ajuda a fazer chamadas <code>**HTTP**</code> a partir de um terminal (**<code>CLI</code> é a abreviatura de <code>command line interface, ou interface de linha de comando</code>**).

### Resumo dos principais repositórios

Quando criarmos uma visualização para esses dados, vamos querer incluir mais de um repositório. Escreveremos um laço para exibir informações selecionadas sobre cada um dos repositórios devolvidos pela chamada de API para que possamos incluir todos eles na visualização.

In [53]:
#python_repos.py
import requests

# Faz uma chamada de API e armazena a resposta 
url = 'https://api.github.com/search/repositories?q=language:python&sort=stars' 
r = requests.get(url) 
print("Status code:", r.status_code)

# Armazena a resposta da API em uma variável 
response_dict = r.json() 
print("Total repositories:", response_dict['total_count'])

# Explora informações sobre os repositórios  
repo_dicts = response_dict['items']
print("Repositories returned:", len(repo_dicts))
print("\nSelected information about each repository:") 
for repo_dict in repo_dicts: 
    print('\nName:', repo_dict['name']) 
    print('Owner:', repo_dict['owner']['login']) 
    print('Stars:', repo_dict['stargazers_count'])
    print('Repository:', repo_dict['html_url']) 
    print('Description:', repo_dict['description'])

# Analisa o primeiro repositório 
repo_dict = repo_dicts[0]
print("\nSelected information about first repository:") 
print('Name:', repo_dict['name']) 
print('Owner:', repo_dict['owner']['login']) 
print('Stars:', repo_dict['stargazers_count']) 
print('Repository:', repo_dict['html_url']) 
print('Created:', repo_dict['created_at']) 
print('Updated:', repo_dict['updated_at']) 
print('Description:', repo_dict['description'])

for key in sorted(repo_dict.keys()): 
    print(key)

Status code: 200
Total repositories: 15916065
Repositories returned: 30

Selected information about each repository:

Name: system-design-primer
Owner: donnemartin
Stars: 266405
Repository: https://github.com/donnemartin/system-design-primer
Description: Learn how to design large-scale systems. Prep for the system design interview.  Includes Anki flashcards.

Name: awesome-python
Owner: vinta
Stars: 214922
Repository: https://github.com/vinta/awesome-python
Description: An opinionated list of awesome Python frameworks, libraries, software and resources.

Name: Python
Owner: TheAlgorithms
Stars: 182983
Repository: https://github.com/TheAlgorithms/Python
Description: All Algorithms implemented in Python

Name: AutoGPT
Owner: Significant-Gravitas
Stars: 165535
Repository: https://github.com/Significant-Gravitas/AutoGPT
Description: AutoGPT is the vision of accessible AI for everyone, to use and to build on. Our mission is to provide the tools, so that you can focus on what matters.

Name:

Exibimos uma mensagem introdutória. E percorremos todos os dicionários em <code>repo_dicts</code> com um laço. **Nesse laço exibimos o <code>nome de cada projeto, o seu proprietário, quantas estrelas o projeto recebeu, seu URL no GitHub</code> e a sua descrição.**

**Alguns projetos interessantes aparecem nesse resultado, e pode valer a pena dar uma olhada em alguns.** No entanto, não gaste muito tempo nisso, pois estamos prestes a criar uma visualização que facilitará bastante a leitura dos resultados.

### Monitorando os limites da taxa de uso da <code>**API**</code>

**A maioria das <code>APIs</code> tem uma taxa de uso limitada, o que significa que há um limite para quantas requisições podemos fazer em determinado período de tempo.** Para ver se estamos nos aproximando dos limites do <code>**GitHub**</code>, forneça o endereço https://api.github.com/rate_limit em um navegador web. Você deverá ver uma resposta como esta:

**"resources": {**   
**"core": { "limit": 60, "remaining": 58, "reset": 1426082320 },    
"search": {  "limit": 10, "remaining": 8, "reset": 1426078803 }},   
"rate": { "limit": 60, "remaining": 58, "reset": 1426082320 }}**

**A informação em que estamos interessados é o limite da taxa de uso da <code>API de pesquisa</code>. E vemos que o limite é de dez requisições por minuto e que temos oito requisições restantes para o minuto atual. O valor de <code>reinicialização (reset)</code> representa o instante na <code>Era Unix (Unix time) ou em epoch time</code>**(o número de segundos desde a meia-noite de 1 de janeiro de 1970) em que nossa quota será reiniciada. Se atingir sua quota, você obterá uma resposta breve que permitirá saber que o limite da <code>**API**</code> foi atingido. Se alcançar o limite, basta esperar até que sua quota seja reiniciada.

**NOTA**   
Muitas <code>**APIs**</code> exigem que você se registre e obtenha uma chave para fazer chamadas de <code>**API**</code>. Na época em que este texto foi escrito, o <code>**GitHub**</code> não tinha esse requisito, porém, se você adquirir uma chave de <code>**API**</code>, seus limites serão bem maiores.

### Visualizando os repositórios usando o <code>**Pygal**</code>

Agora que temos alguns dados interessantes, **vamos criar uma visualização que mostre a popularidade relativa dos projetos Python no <code>GitHub</code>. Criaremos um <code>gráfico de barras interativo</code>**: a altura de cada barra representará o número de estrelas que o projeto recebeu. Clicar em uma barra levará você para a página inicial do projeto no <code>**GitHub**</code>. A seguir, temos uma tentativa inicial:

In [8]:
#python_repos.py 
import requests 
import pygal 
from pygal.style import LightColorizedStyle as LCS, LightenStyle as LS

# Faz uma chamada de API e armazena a resposta 
URL = 'https://api.github.com/search/repositories?q=language:python&sort=star' 
r = requests.get(URL) 
print("Status code:", r.status_code)

#Armazena a resposta da API em uma variável 
response_dict = r.json() 
print("Total repositories:", response_dict['total_count'])

# Explora informações sobre os repositórios 
repo_dicts = response_dict['items']

names, stars = [], []
for repo_dict in repo_dicts: 
    names.append(repo_dict['name']) 
    stars.append(repo_dict['stargazers_count'])

# Cria a visualização 
my_style = LS('#333366', base_style=LCS) 
chart = pygal.Bar(style=my_style, x_label_rotation=45, show_legend=False) 
chart.title = 'Most-Starred Python Projects on GitHub'
chart.x_labels = names
chart.add('', stars) 
chart.render_to_file('python_repos.svg')

Status code: 200
Total repositories: 16327656


**Começamos importando o <code>pygal</code> e os estilos do <code>pygal</code>  de que precisaremos para o gráfico.** Continuamos exibindo o status da resposta da chamada à <code>**API**</code> e o número total de repositórios encontrados para que possamos saber caso haja algum problema com a chamada da <code>**API**</code>. Não exibimos mais as informações sobre os projetos específicos devolvidos, pois elas serão incluídas na visualização.

**E criamos duas <code>listas vazias</code> para armazenar os dados que incluiremos no gráfico.** Precisaremos do nome de cada projeto para rotular as barras e do número de estrelas para determinar a altura delas. **No laço, concatenamos nessas listas o nome de cada projeto e o número de estrelas que ele tem.**

**Em seguida, definimos um estilo usando a classe <code>LightenStyle (alias LS)</code> e usamos um tom de 'azul-escuro' como base. Também passamos o argumento <code>base_style</code> para utilizar a classe <code>LightenStyle (alias LS)</code>. Então usamos <code>Bar()</code> para criar um gráfico de barras simples e lhe passamos <code>my_style</code>.** Além disso, passamos outros dois argumentos de estilo: definimos a rotação dos nomes ao longo do <code>**eixo x em 45 graus (x_label_rotation=45)**</code> e ocultamos a <code>**legenda**</code>, pois estamos plotando apenas uma série no gráfico <code>**(show_legend=False)**</code>. **Então fornecemos um título ao gráfico e definimos o atributo <code>x_labels</code> com a lista <code>names</code>**.

**Como não é necessário nomear essa série de dados, passamos uma <code>string vazia</code> para o rótulo quando adicionamos os dados.** O gráfico resultante pode ser visto na Figura 17.1. Podemos ver que os primeiros projetos são significativamente mais populares que os demais, mas todos eles são importantes no ecossistema de Python.

### Aperfeiçoando os gráficos do **<code>pygal</code>**

**Vamos melhorar a estilização de nosso gráfico.** Faremos algumas personalizações diferentes, portanto, em primeiro lugar, reestruture um pouco o código criando um objeto de configuração que contenha todas as nossas personalizações, para que seja passado para <code>**Bar()**</code>: 

In [15]:
#python_repos.py 
import requests 
import pygal 
from pygal.style import LightColorizedStyle as LCS, LightenStyle as LS

# Faz uma chamada de API e armazena a resposta 
URL = 'https://api.github.com/search/repositories?q=language:python&sort=star' 
r = requests.get(URL) 
print("Status code:", r.status_code)

#Armazena a resposta da API em uma variável 
response_dict = r.json() 
print("Total repositories:", response_dict['total_count'])

# Explora informações sobre os repositórios 
repo_dicts = response_dict['items']

names, stars = [], []
for repo_dict in repo_dicts: 
    names.append(repo_dict['name']) 
    stars.append(repo_dict['stargazers_count'])

# Cria a visualização 
my_style = LS('#333366', base_style=LCS)
my_config = pygal.Config() 
my_config.x_label_rotation = 45
my_config.show_legend = False 
my_config.title_font_size = 24 
my_config.label_font_size = 14
my_config.major_label_font_size = 18
my_config.truncate_label = 15
my_config.show_y_guides = False 
my_config.width = 1000
chart = pygal.Bar(my_config, style=my_style)

chart.title = 'Most-Starred Python Projects on GitHub'
chart.x_labels = names
chart.add('', stars) 
chart.render_to_file('python_repos1.svg')

Status code: 200
Total repositories: 15786789


E criamos **uma instância chamada <code>my_config</code> da classe Config do Pygal<code></code>; modificar os atributos de <code>my_config</code> personalizará a aparência do gráfico. Definimos os dois atributos <code>x_label_rotation e show_legend</code>, originalmente passados como argumentos nomeados quando criamos uma instância de <code>Bar</code>. E definimos o tamanho da fonte para o título do gráfico e para os rótulos menores e maiores.** Os rótulos menores nesse gráfico são os nomes dos projetos ao longo do <code>**eixo x**</code> e a maior parte dos números no <code>**eixo y**</code>. Os rótulos maiores são apenas os rótulos do <code>**eixo y**</code> que marcam incrementos de 5.000 estrelas. Esses rótulos serão maiores, e é por isso que os diferenciamos. **E usamos <code>truncate_label</code> para reduzir os nomes de projeto mais longos a 15 caracteres.** (Quando você passar o mouse sobre um nome de projeto truncado na tela, o nome completo aparecerá.) **Em seguida, ocultamos as linhas horizontais do gráfico definindo <code>show_y_guides com False</code>**. Por fim, definimos uma largura personalizada para que o gráfico use mais do espaço disponível no navegador.

**Agora, quando criamos uma instância de <code>Bar at</code>, passamos <code>my_config</code> como primeiro argumento, e todas as nossas definições de configuração serão enviadas em um só argumento.** Podemos fazer quantas modificações de estilo e de configuração quisermos por meio de <code>**my_config**</code>, e a linha e não mudará. A Figura 17.2 mostra o gráfico reestilizado.

### Acrescentando dicas de contexto personalizadas

**No <code>Pygal</code>, passar o cursor sobre uma barra individual faz com que as informações representadas pela barra sejam exibidas.** Elas são comumente chamadas de **<code>dicas de contexto (tooltips)</code> e, nesse caso, mostram o número de estrelas que um projeto tem. Criaremos uma <code>dica de contexto</code> personalizada que mostre também a descrição de cada projeto.**

**Vamos ver um pequeno exemplo que usa os três primeiros projetos plotados individualmente, com rótulos personalizados passados para cada barra**. Para isso passaremos uma lista de dicionários para **<code>add()</code>** no lugar de uma lista de valores:

In [20]:
#bar_descriptions.py 
import pygal 
from pygal.style import LightColorizedStyle as LCS, LightenStyle as LS

my_style = LS('#333366', base_style=LCS) 
chart = pygal.Bar(style=my_style, x_label_rotation=45, show_legend=False) 
chart.title = 'Python Projects'
chart.x_labels = ['httpie', 'django', 'flask']
plot_dicts = [{'value': 16101, 'label': 'Description of httpie.'}, 
              {'value': 15028, 'label': 'Description of django.'}, 
              {'value': 14798, 'label': 'Description of flask.'}, ]
chart.add('', plot_dicts) 
chart.render_to_file('bar_descriptions.svg')

**E definimos uma lista chamada <code>plot_dicts</code>, que contém três dicionários: um para o projeto <code>HTTPie</code>, um para o projeto <code>Django</code> e outro para o <code>Flask</code>.** Cada dicionário tem duas chaves: **<code>'value' e 'label'</code>. O <code>Pygal</code> usa o número associado a <code>'value'</code> para descobrir a altura que cada barra deve ter, e utiliza a string associada a <code>'label'</code> para criar a <code>dica de contexto</code> de cada barra.** Por exemplo, o primeiro dicionário criará uma barra que representa um projeto com 16.101 estrelas, e sua <code>**dica de contexto**</code> conterá <code>**Description of httpie (Descrição de httpie)**</code>.

**O método <code>add()</code> precisa de uma string e de uma lista. Quando chamamos esse método, passamos a lista de dicionários que representa as barras <code>(plot_dicts)</code>.** A Figura 17.3 mostra uma das dicas de contexto. O <code>**Pygal**</code> inclui o número de estrelas como uma <code>**dica de contexto default**</code>, além da dica de contexto personalizada que lhe passamos.

### Plotando os dados

**Para plotar nossos dados, vamos gerar <code>(plot_dicts)</code> automaticamente para os 30 projetos devolvidos pela chamada de <code>API</code>.**   
Eis o código para fazer isso:

In [31]:
#python_repos.py 
import requests 
import pygal 
from pygal.style import LightColorizedStyle as LCS, LightenStyle as LS

# Faz uma chamada de API e armazena a resposta 
URL = 'https://api.github.com/search/repositories?q=language:python&sort=star' 
r = requests.get(URL) 
print("Status code:", r.status_code)

#Armazena a resposta da API em uma variável 
response_dict = r.json() 
print("Total repositories:", response_dict['total_count'])

# Explora informações sobre os repositórios 
repo_dicts = response_dict['items']
print("Number of items:", len(repo_dicts))

names, plot_dicts = [], []
for repo_dict in repo_dicts: 
    names.append(repo_dict['name'])
    plot_dict = {'value': repo_dict['stargazers_count'], 'label':repo_dict['description'], } 
    plot_dicts.append(plot_dict)
    
# Cria a visualização 
my_style = LS('#333366', base_style=LCS)
my_config = pygal.Config() 
my_config.x_label_rotation = 45
my_config.show_legend = False 
my_config.title_font_size = 24 
my_config.label_font_size = 14
my_config.major_label_font_size = 18
my_config.truncate_label = 15
my_config.show_y_guides = False 
my_config.width = 1000
chart = pygal.Bar(my_config, style=my_style)

chart.title = 'Most-Starred Python Projects on GitHub'
chart.x_labels = names
chart.add('', stars)
chart.add('', plot_dicts)
chart.render_to_file('python_repos2.svg')

Status code: 200
Total repositories: 16065853
Number of items: 30


**E criamos uma lista vazia para names e outra para <code>(plot_dicts)</code>. Ainda precisamos da lista <code>names</code> para gerar os rótulos do <code>eixo x</code>.**
**No laço criamos o dicionário <code>plot_dict</code> para cada projeto. Armazenamos o número de estrelas com a chave <code>'value'</code> e a descrição do projeto com a chave <code>'label'</code> em cada <code>plot_dict</code>.** Então concatenamos o <code>**plot_dict**</code> de cada projeto em <code>**plot_dicts**</code>.     
E passamos a lista <code>**plot_dicts para add()**</code> . A Figura 17.4 mostra o gráfico resultante.

### Adicionando <code>**links**</code> que podem ser clicados em nosso gráfico

O **<code>Pygal</code> também permite usar cada barra do gráfico como um <code>link</code> para um site. Para acrescentar essa funcionalidade, basta adicionar uma linha em nosso código, tirando proveito do dicionário que criamos para cada projeto.** Adicionamos um novo <code>**par chave-valor ao plot_dict**</code> de cada projeto usando a <code>**chave 'xlink'**</code>:

In [38]:
#python_repos.py 
import requests 
import pygal 
from pygal.style import LightColorizedStyle as LCS, LightenStyle as LS

# Faz uma chamada de API e armazena a resposta 
URL = 'https://api.github.com/search/repositories?q=language:python&sort=star' 
r = requests.get(URL) 
print("Status code:", r.status_code)

#Armazena a resposta da API em uma variável 
response_dict = r.json() 
print("Total repositories:", response_dict['total_count'])

# Explora informações sobre os repositórios 
repo_dicts = response_dict['items']
print("Number of items:", len(repo_dicts))

names, plot_dicts = [], []
for repo_dict in repo_dicts: 
    names.append(repo_dict['name'])
    plot_dict = {'value': repo_dict['stargazers_count'], 'label':repo_dict['description'], 'xlink': repo_dict['html_url'],} 
    plot_dicts.append(plot_dict)
    
# Cria a visualização 
my_style = LS('#333366', base_style=LCS)
my_config = pygal.Config() 
my_config.x_label_rotation = 45
my_config.show_legend = False 
my_config.title_font_size = 24 
my_config.label_font_size = 14
my_config.major_label_font_size = 18
my_config.truncate_label = 15
my_config.show_y_guides = False 
my_config.width = 1000
chart = pygal.Bar(my_config, style=my_style)

chart.title = 'Most-Starred Python Projects on GitHub'
chart.x_labels = names
chart.add('', stars)
chart.add('', plot_dicts)
chart.render_to_file('python_repos2.svg')

Status code: 200
Total repositories: 16317343
Number of items: 30


O **<code>Pygal</code> usa o <code>URL</code> associado a <code>'xlink'</code> para transformar cada barra em um <code>link ativo</code>. Você pode clicar em qualquer uma das barras do gráfico e a página desse projeto no <code>GitHub</code> será automaticamente aberta em uma nova aba em seu navegador.** Agora você tem uma visualização informativa e interativa dos dados obtidos por meio de uma <code>**API**</code>! 

### A **API de Hacker News**

Para explorar o uso de chamadas de <code>**API**</code> em outros sites, daremos uma olhada em <code>**Hacker News**</code> (http://news.ycombinator.com/). **No <code>Hacker News</code>, as pessoas compartilham artigos sobre programação e tecnologia, e se envolvem em discussões entusiasmadas sobre esses artigos. A <code>API do Hacker News</code> oferece acesso a dados sobre todos os artigos submetidos e os comentários do site, disponíveis sem a necessidade de se registrar para obter uma chave.**    
A chamada a seguir devolve informações sobre os principais artigos do momento (na ocasião em que este livro foi escrito):
https://hacker- news.firebaseio.com/v0/item/9884165.json    

A resposta é um dicionário com informações sobre o artigo cujo ID é 9884165:    
**{     
u 'url': 'http://www.bbc.co.uk/news/science-environment-33524589', 'type':    
'story', v 'title': 'New Horizons: Nasa spacecraft speeds past Pluto', w 'descendants': 141, 'score': 230, 'time':    1436875181, 'text': '', 'by': 'nns', 'id': 9884165, x 'kids': [9884723, 9885099, 9884789, 9885604, 9885844]
}**

O dicionário contém várias chaves com as quais podemos trabalhar, por exemplo, <code>**'url' e 'title'**</code>. **A chave '<code>descendants'</code> contém a quantidade de comentários que um artigo recebeu . A chave <code>'kids'</code> oferece os <code>IDs</code> de todos os comentários feitos diretamente em resposta a esse artigo submetido.** Cada um desses comentários pode ter filhos próprios também, portanto o número de descendentes que um artigo submetido tem pode ser maior que o número de seus filhos.

**Vamos fazer uma chamada de <code>API</code> que devolva os <code>IDs</code> dos principais artigos do momento no <code>Hacker News</code> e então analisaremos cada um desses artigos:**

In [57]:
import requests
from operator import itemgetter

# Faz uma chamada de API e armazena a resposta 
url = 'https://hacker-news.firebaseio.com/v0/topstories.json'
r = requests.get(url) 
print("Status code:", r.status_code)

# Processa informações sobre cada artigo submetido 
submission_ids = r.json() 
submission_dicts = []
for submission_id in submission_ids[:30]: 
    # Cria uma chamada de API separada para cada artigo submetido 
    url = ('https://hacker-news.firebaseio.com/v0/item/' + str(submission_id) + '.json') 
    submission_r = requests.get(url) 
    print(submission_r.status_code) 
    response_dict = submission_r.json()
    submission_dict = {'title': response_dict['title'], 'link': 'http://news.ycombinator.com/item?id=' + str(submission_id), 
                       'comments': response_dict.get('descendants', 0) }
    submission_dicts.append(submission_dict)

# Ordena a lista fora do loop
submission_dicts = sorted(submission_dicts, key=itemgetter('comments'), reverse=True)

for submission_dict in submission_dicts: 
    print("\nTitle:", submission_dict['title']) 
    print("Discussion link:", submission_dict['link']) 
    print("Comments:", submission_dict['comments'])

Status code: 200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200

Title: In ‘The Book Against Death,’ Elias Canetti rants against mortality
Discussion link: http://news.ycombinator.com/item?id=41186117
Comments: 425

Title: NASA Says Boeing Starliner Astronauts May Fly Home on SpaceX in 2025
Discussion link: http://news.ycombinator.com/item?id=41184359
Comments: 405

Title: Prevention of HIV
Discussion link: http://news.ycombinator.com/item?id=41184365
Comments: 243

Title: RLHF is just barely RL
Discussion link: http://news.ycombinator.com/item?id=41188647
Comments: 196

Title: More and more German trains are not allowed to enter Switzerland
Discussion link: http://news.ycombinator.com/item?id=41189834
Comments: 181

Title: How French Drains Work
Discussion link: http://news.ycombinator.com/item?id=41176461
Comments: 176

Title: Firefox Sidebar and Vertical tabs: try them out in Nightly Firefox Labs 131
Discussion

**Inicialmente fizemos a chamada de <code>API</code> e exibimos o <code>status da resposta</code>. Essa chamada de <code>API</code> devolve uma lista contendo os <code>IDs dos 500</code>  artigos mais populares do <code>Hacker News</code> no momento em que a chamada foi feita. Então convertemos o texto da resposta em uma lista Python, que armazenamos em <code>submission_ids</code>.** Usaremos esses <code>**IDs**</code> para criar um conjunto de <code>**dicionários**</code> em que cada um armazenará informações sobre um dos artigos submetidos.

Criamos **uma <code>**lista vazia**</code> chamada <code>submission_dicts</code> para armazenar esses dicionários. Então percorremos os <code>IDs dos 30</code> principais artigos submetidos com um laço. Fazemos uma nova chamada de <code>API</code> para cada artigo gerando um <code>URL</code> que inclui o valor atual de <code>submission_id</code>.** Exibimos o status<code></code> de cada requisição para que possamos ver se ela foi bem-sucedida.

**E criamos um dicionário para o artigo submetido processado no momento, no qual armazenamos o <code>título do artigo e um link</code> para a página de discussão desse item**. E armazenamos o número de <code>**comentários**</code> no dicionário. Se um artigo ainda não teve nenhum comentário, a <code>**chave 'descendants'**</code> não estará presente. **Quando você não tiver certeza de que uma chave existe em um dicionário, utilize o método <code>dict.get()</code>, que devolve o valor associado à chave especificada se ela existir, ou o valor que você fornecer se ela não existir (0 nesse exemplo)**. Por fim, concatenamos cada <code>**submission_dict à lista submission_dicts**</code>.

Os artigos submetidos no <code>**Hacker News**</code> são classificados de acordo com uma pontuação geral, baseada em vários fatores, incluindo quantos votos receberam, quantos comentários foram feitos e quão recentemente o artigo foi submetido.   Queremos ordenar a lista de dicionários de acordo com o número de <code>**comentários**</code>. Para isso, usamos uma função chamada **<code>itemgetter()</code>, proveniente do módulo <code>operator</code>. Passamos a chave <code>'comments'</code> a essa função e ela extrai o valor associado a essa chave de cada dicionário da lista. A função <code>sorted()</code> então utiliza esse valor como base para ordenar a lista.** Ordenamos a lista na ordem inversa para colocar as histórias mais comentadas antes.

Depois que a lista estiver ordenada, nós a percorremos com um laço e exibimos três informações sobre cada um dos principais artigos submetidos: **o título, um link para a página de discussão e o número de comentários** que o artigo submetido tem no momento: **Status code: 200     
200    
200     
200     
--trecho omitido--**


**Você poderia usar um processo semelhante para acessar e analisar informações com qualquer <code>API</code>**. Com esses dados você poderia criar uma visualização que mostre quais artigos submetidos inspiraram as discussões recentes mais entusiasmadas.

## **FAÇA VOCÊ MESMO**

**17.1 – Outras linguagens:** Modifique a chamada de API em python_repos.py para que ela gere um gráfico mostrando os projetos mais populares em outras linguagens. Experimente usar linguagens como JavaScript, Ruby, C, Java, Perl, Haskell e Go.

In [70]:
import requests 
import pygal 
from pygal.style import LightColorizedStyle as LCS, LightenStyle as LS

# Lista de linguagens para buscar os projetos mais populares
languages = ['python', 'javascript', 'ruby', 'c', 'java', 'perl', 'haskell', 'go']

# Cabeçalhos para a API do GitHub
headers = {'Accept': 'application/vnd.github.v3+json'}

for language in languages:
    # Faz uma chamada de API e armazena a resposta 
    URL = 'https://api.github.com/search/repositories?q=language:' + language + '&sort=stars'
    r = requests.get(URL, headers=headers) 
    print("Status code for " + language + ":", r.status_code)

    # Armazena a resposta da API em uma variável 
    response_dict = r.json() 
    print("Total repositories for " + language + ":", response_dict['total_count'])

    # Explora informações sobre os repositórios 
    repo_dicts = response_dict['items']
    print("Number of items for " + language + ":", len(repo_dicts))

    names, plot_dicts = [], []
    for repo_dict in repo_dicts: 
        names.append(repo_dict['name'])
        plot_dict = {
            'value': repo_dict['stargazers_count'], 
            'label': repo_dict['description'], 
            'xlink': repo_dict['html_url']
        }
        plot_dicts.append(plot_dict)
    
    # Cria a visualização 
    my_style = LS('#333366', base_style=LCS)
    my_config = pygal.Config() 
    my_config.x_label_rotation = 45
    my_config.show_legend = False 
    my_config.title_font_size = 24 
    my_config.label_font_size = 14
    my_config.major_label_font_size = 18
    my_config.truncate_label = 15
    my_config.show_y_guides = False 
    my_config.width = 1000
    chart = pygal.Bar(my_config, style=my_style)

    chart.title = 'Most-Starred ' + language.capitalize() + ' Projects on GitHub'
    chart.x_labels = names
    chart.add('', plot_dicts)
    file_name = language + '_repos.svg'
    chart.render_to_file(file_name)

    print("Chart for " + language + " saved as " + file_name)

Status code for python: 200
Total repositories for python: 15854507
Number of items for python: 30
Chart for python saved as python_repos.svg
Status code for javascript: 200
Total repositories for javascript: 26522907
Number of items for javascript: 30
Chart for javascript saved as javascript_repos.svg
Status code for ruby: 200
Total repositories for ruby: 2879254
Number of items for ruby: 30
Chart for ruby saved as ruby_repos.svg
Status code for c: 200
Total repositories for c: 3125970
Number of items for c: 30
Chart for c saved as c_repos.svg
Status code for java: 200
Total repositories for java: 16253782
Number of items for java: 30
Chart for java saved as java_repos.svg
Status code for perl: 200
Total repositories for perl: 180120
Number of items for perl: 30
Chart for perl saved as perl_repos.svg
Status code for haskell: 200
Total repositories for haskell: 146231
Number of items for haskell: 30
Chart for haskell saved as haskell_repos.svg
Status code for go: 200
Total repositories

**17.2 – Discussões entusiasmadas:** Usando os dados de hn_submissions.py, crie um gráfico de barras que mostre as discussões mais entusiasmadas do momento no Hacker News. A altura de cada barra deve corresponder ao número de comentários que cada artigo submetido tem. O rótulo de cada barra deve incluir o título do artigo submetido, e cada barra deve atuar como um link para a página de discussão desse artigo.

In [72]:
import requests
import pygal
from pygal.style import LightColorizedStyle as LCS, LightenStyle as LS

# Faz uma chamada de API e armazena a resposta 
url = 'https://hacker-news.firebaseio.com/v0/topstories.json'
r = requests.get(url)
print("Status code:", r.status_code)

# Processa informações sobre cada artigo submetido 
submission_ids = r.json()
submission_dicts = []
for submission_id in submission_ids[:30]: 
    # Cria uma chamada de API separada para cada artigo submetido 
    url = ('https://hacker-news.firebaseio.com/v0/item/' + str(submission_id) + '.json') 
    submission_r = requests.get(url)
    response_dict = submission_r.json()
    submission_dict = {
        'title': response_dict['title'], 
        'link': 'http://news.ycombinator.com/item?id=' + str(submission_id), 
        'comments': response_dict.get('descendants', 0)
    }
    submission_dicts.append(submission_dict)

# Ordena a lista de artigos pela quantidade de comentários
submission_dicts = sorted(submission_dicts, key=lambda x: x['comments'], reverse=True)

# Preparação dos dados para o gráfico
titles = []
plot_dicts = []
for submission_dict in submission_dicts:
    titles.append(submission_dict['title'])
    plot_dict = {
        'value': submission_dict['comments'],
        'label': submission_dict['title'],
        'xlink': submission_dict['link'],
    }
    plot_dicts.append(plot_dict)

# Configurações do gráfico
my_style = LS('#333366', base_style=LCS)
my_config = pygal.Config()
my_config.x_label_rotation = 45
my_config.show_legend = False
my_config.title_font_size = 24
my_config.label_font_size = 14
my_config.major_label_font_size = 18
my_config.truncate_label = 15
my_config.show_y_guides = False
my_config.width = 1000

# Criação do gráfico
chart = pygal.Bar(my_config, style=my_style)
chart.title = 'Most Active Discussions on Hacker News'
chart.x_labels = titles
chart.add('', plot_dicts)
chart.render_to_file('hn_discussions.svg')

print("Chart saved as hn_discussions.svg")

Status code: 200
Chart saved as hn_discussions.svg


**17.3 – Testando python_repos.py:** Em **python_repos.py**, exibimos o valor de **status_code** para garantir que a chamada de API foi bem-sucedida. Escreva um programa chamado test_python_repos.py que use *unittest** para conferir se o valor de **status_code** é 200. Descubra outras asserções que você possa fazer – por exemplo, se o número de itens devolvidos é o que se espera e se o número total de repositórios é maior que uma determinada quantidade.


In [80]:
import unittest
import requests

class TestPythonRepos(unittest.TestCase):
    """Testes para a API do GitHub e os dados retornados."""

    def setUp(self):
        """Faz a chamada à API e armazena a resposta."""
        self.url = 'https://api.github.com/search/repositories?q=language:python&sort=stars'
        self.headers = {'Accept': 'application/vnd.github.v3+json'}
        self.response = requests.get(self.url, headers=self.headers)
        self.response_dict = self.response.json()
    
    def test_status_code(self):
        """Testa se a chamada à API foi bem-sucedida."""
        self.assertEqual(self.response.status_code, 200)
    
    def test_items_returned(self):
        """Testa se o número de repositórios retornados é o esperado."""
        self.assertEqual(len(self.response_dict['items']), 30)
    
    def test_total_repositories(self):
        """Testa se o número total de repositórios é maior que um determinado valor."""
        self.assertGreater(self.response_dict['total_count'], 70000)

# Executa os testes manualmente
suite = unittest.TestLoader().loadTestsFromTestCase(TestPythonRepos)
unittest.TextTestRunner(verbosity=2).run(suite)

test_items_returned (__main__.TestPythonRepos.test_items_returned)
Testa se o número de repositórios retornados é o esperado. ... ok
test_status_code (__main__.TestPythonRepos.test_status_code)
Testa se a chamada à API foi bem-sucedida. ... ok
test_total_repositories (__main__.TestPythonRepos.test_total_repositories)
Testa se o número total de repositórios é maior que um determinado valor. ... ok

----------------------------------------------------------------------
Ran 3 tests in 2.079s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

****

## **Resumo**

Neste capítulo aprendemos a usar <code>**APIs**</code> para escrever programas autocontidos que coletem automaticamente os dados necessários e usem esses dados para criar uma visualização. **Usamos a <code>API do GitHub</code> para explorar os projetos Python com mais estrelas no <code>GitHub</code>e vimos rapidamente a <code>API do Hacker News</code> também. Aprendemos a usar o pacote <code>requests</code> para fazer uma chamada de <code>API do GitHub</code> de modo automático e a processar os resultados dessa chamada.** Também apresentamos algumas configurações do <code>**Pygal*</code> para personalizar melhor a aparência dos gráficos que você gerar.

No último projeto, usaremos <code>**Django**</code> para criar uma <code>**aplicação web**</code>.